# Lab 2 - Travelling salesman problem
https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [36]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random
from itertools import combinations
from tqdm.auto import tqdm

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

/home/lorenzo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Loading

In [63]:
SOURCE = 'cities/vanuatu.csv'
CITIES = pd.read_csv(SOURCE, header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Isangel,-19.53,169.28
1,Lakatoro,-16.09,167.40
2,Longana,-15.30,168.00
3,Luganville,-15.51,167.15
4,Norsup,-16.07,167.39


### Fitness

In [22]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

### Faster but less accurate version

In [64]:
POPULATION_SIZE = 200
NUM_GENERATIONS = 200
MUTATION_RATE = 0.2

def initialize_population(pop_size, num_cities):
    population = [np.random.permutation(num_cities).tolist() for _ in range(pop_size)]
    for tour in population:
        tour.append(tour[0])  
    return population

def tournament_selection(population, k=10):
    selected = random.choices(population, k=k)
    selected_fitness = [tsp_cost(tour) for tour in selected]
    return selected[np.argmin(selected_fitness)]

# Partially Mapped Crossover (PMX) for creating offspring
def pmx_crossover(parent1, parent2):
    size = len(parent1) - 1
    start, end = sorted(random.sample(range(size), 2))
    child = [-1] * size
    child[start:end+1] = parent1[start:end+1]

    for i in range(start, end+1):
        if parent2[i] not in child:
            idx = i
            while start <= idx <= end:
                idx = parent2.index(parent1[idx])
            child[idx] = parent2[i]

    for i in range(size):
        if child[i] == -1:
            child[i] = parent2[i]

    return child + [child[0]]

def mutate(tour, rate = MUTATION_RATE):
    if random.random() < rate:
        idx1, idx2 = random.sample(range(1, len(tour) - 2), 2)
        tour[idx1], tour[idx2] = tour[idx2], tour[idx1]
    return tour

def evolutionary_algorithm():
    population = initialize_population(POPULATION_SIZE, len(CITIES))
    best_tour = min(population, key=tsp_cost)
    best_fitness = tsp_cost(best_tour)

    for generation in tqdm(range(NUM_GENERATIONS)):
        new_population = []
        for _ in range(POPULATION_SIZE // 2):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child1 = mutate(pmx_crossover(parent1, parent2))
            child2 = mutate(pmx_crossover(parent2, parent1))
            new_population.extend([child1, child2])

        population = sorted(new_population + population, key=tsp_cost)[:POPULATION_SIZE]
        current_best = min(population, key=tsp_cost)
        if tsp_cost(current_best) < best_fitness:
            best_tour = current_best
            best_fitness = tsp_cost(current_best)

    return best_tour, best_fitness

best_tour_result, best_fitness_result = evolutionary_algorithm()
print(f"Best Tour:{best_tour_result}")
print(f"Best Fitness (Total Distance): {best_fitness_result:.2f}km")

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:03<00:00, 52.68it/s]


Best Tour:[4, 3, 5, 6, 2, 0, 7, 1, 4]
Best Fitness (Total Distance): 1345.54km


### Slow but more accurate version

In [65]:
POPULATION_SIZE = 200
NUM_GENERATIONS = 1000
MUTATION_RATE = 0.3
ELITE_FRACTION = 0.02

def initialize_population(pop_size, num_cities):
    population = []
    for _ in range(pop_size):
        new_tour = np.random.permutation(num_cities).tolist()
        while new_tour in population:
            new_tour = np.random.permutation(num_cities).tolist()
        population.append(new_tour)
    for tour in population:
        tour.append(tour[0])
    return population

def roulette_wheel_selection(population, fitness_scores):
    fitness_total = sum(1 / f for f in fitness_scores)
    probabilities = [(1 / f) / fitness_total for f in fitness_scores]
    selected_index = np.random.choice(len(population), p=probabilities)
    return population[selected_index]

# Crossover: Order Crossover (OX) for more diverse offspring
def order_crossover(parent1, parent2):
    size = len(parent1) - 1
    start, end = sorted(random.sample(range(size), 2))
    
    child = [-1] * size
    child[start:end+1] = parent1[start:end+1]

    fill_positions = [i for i in range(size) if child[i] == -1]
    fill_values = [city for city in parent2 if city not in child]
    for i, value in zip(fill_positions, fill_values):
        child[i] = value
    
    return child + [child[0]]

def adaptive_mutate(tour, generation, num_gens = NUM_GENERATIONS, rate = MUTATION_RATE):
    adjusted_rate = rate * (1 - (generation / num_gens))
    if random.random() < adjusted_rate:
        idx1, idx2 = random.sample(range(1, len(tour) - 2), 2)
        tour[idx1], tour[idx2] = tour[idx2], tour[idx1]
    return tour

def evolutionary_algorithm():
    population = initialize_population(POPULATION_SIZE, len(CITIES))
    best_tour = min(population, key=tsp_cost)
    best_fitness = tsp_cost(best_tour)

    for generation in tqdm(range(NUM_GENERATIONS)):
        fitness_scores = [tsp_cost(tour) for tour in population]
        elite_count = int(ELITE_FRACTION * POPULATION_SIZE)
        elites = sorted(population, key=tsp_cost)[:elite_count]
        
        new_population = elites[:]
        for _ in range(POPULATION_SIZE - len(new_population)):
            parent1 = roulette_wheel_selection(population, fitness_scores)
            parent2 = roulette_wheel_selection(population, fitness_scores)
            
            child = adaptive_mutate(order_crossover(parent1, parent2), generation)
            new_population.append(child)

        population = new_population
        current_best = min(population, key=tsp_cost)
        if tsp_cost(current_best) < best_fitness:
            best_tour, best_fitness = current_best, tsp_cost(current_best)

    return best_tour, best_fitness

best_tour_result, best_fitness_result = evolutionary_algorithm()
print(f"Best Tour:{best_tour_result}")
print(f"Best Fitness (Total Distance): {best_fitness_result:.2f}km")

100%|██████████| 1000/1000 [01:08<00:00, 14.56it/s]


Best Tour:[4, 3, 5, 6, 2, 0, 7, 1, 4]
Best Fitness (Total Distance): 1345.54km
